In [18]:
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [12]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [16]:
model_name = "gpt-3.5-turbo"
chat = ChatOpenAI(model_name=model_name, temperature=0.7)

# Como fazemos os prompts?
OS prompts são feitos com os espaços das variáveis, servindo como pergunta para a LLM. Uma coisa legal é que tudo é independente, então voce pode usar diferentes modelos para diferentes situações

In [21]:
template = ChatPromptTemplate.from_template("Quanto é {valor} + 2?")
resposta = template.invoke({'valor':3})
print(resposta)


messages=[HumanMessage(content='Quanto é 3 + 2?')]


In [23]:
chat.invoke("Quanto é 3 + 2?")

AIMessage(content='3 + 2 é igual a 5.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 16, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-56170964-1ba7-41cf-af0b-3c89bb5c3e53-0')

# Sequencia de Prompts
Nesse caso usamos o conceito de _runnable interface_, que é basicamente uma sequência de atos para obter um resultado. Um exemplo abaixo será feito no caso de uma viagem. Importante lembrar que o output geralmente não é uma string e nesse caso é muito importante fazer um parse da saída

In [ ]:
template_viagem = ChatPromptTemplate.from_template("Quero viajar para {destino} e gostaria de saber mais informações sobre o local.")
template_dias = ChatPromptTemplate.from_template("Vou ficar {dias} dias e preciso de um roteiro de viagem.")
template_orcamento = ChatPromptTemplate.from_template("Meu orçamento é de R$ {valor} e gostaria de saber se é possível para esse roteiro.")


from langchain.callbacks.tracers import ConsoleCallbackHandler


# nesse caso, a saída do primeiro entra no segundo, e a saída do segundo entra no terceiro
chain = template_viagem | chat | StrOutputParser() | template_dias | chat | StrOutputParser() | template_orcamento | chat | StrOutputParser()
resposta = chain.invoke({'destino':'Paris', 'dias':5, 'valor':5000},config={'callbacks': [ConsoleCallbackHandler()]})


Tem como voce indicar no prompt para ele te dar apenas uma palavra e tudo mais. O langchain nos entrega uma forma de analisar o output e pegar a parte mais importante

In [38]:
from langchain_core.pydantic_v1 import Field, BaseModel

In [44]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.globals import set_debug
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

set_debug(True)

# Utilizando o pydantic, craimos dataclasses e especificamos como queremos nosso output. Esse tipo de script cria no prompt uma forma de receber a resposta a partir de um formato específico, justamente o que a função get_format_instructions() faz.
class CamposSaida(BaseModel):
    cidade: str = Field("Apenas o nome da cidade")
    lista_atracoes: list = Field("Lista com 3 atrações turísticas")

parser = PydanticOutputParser(pydantic_object=CamposSaida)

template_viagem = PromptTemplate(template = "Quero viajar para {destino} e gostaria de saber mais informações sobre o local./n {resposta}",input_variables=['destino'],partial_variables={'resposta':parser.get_format_instructions()})
chain_cidade = template_viagem | chat 
chain_cidade.invoke({'destino':'Paris'})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "destino": "Paris"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "destino": "Paris"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Quero viajar para Paris e gostaria de saber mais informações sobre o local./n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schem

{'cidade': 'Paris',
 'lista_atracoes': ['Torre Eiffel',
  'Museu do Louvre',
  'Catedral de Notre-Dame']}